In [1]:
##### CONFIG

import os


# Carpetas que contienen los audios
AUDIO_FOLDER = "audio_database"
AUDIO_DATABASE = "audios25m_test"

# Carpeta de salida
RESULT_FOLDER = "resultadosBaseDatos"
RESULT_DATABASE = "pruebas_andres_22_oct.csv"


# Windows params in ms
WIN_LENGTH = 25     # windows length
WIN_OVERLAP = 10    # 10 ms of overlapping means it moves (25-10)ms = 15 ms with each step

#https://www.kaggle.com/code/ilyamich/mfcc-implementation-and-tutorial/notebook
#https://librosa.org/doc/latest/index.html


#%% Opción de testeo
#% Cuando se generan los datos de train se debe incluir la clase en la
#% ventana
#IS_TRAINING = true;


#REDUCE_SILENCES = false;   % Si se quiere eliminar silencios (ASICA tiene demasiados)
#SILENT_RATE = 0.2;         % Ratio que queda de silencios

#% Si se quiere realizar la detección de silencios al inicio y al final se
#% pone a true. Si no, a false.
#DETECT_SILENCES = false;


#% Ajuste de las vocales al 30% de las consonantes nasales
#NV_PERCENT_AFTER_BEFORE = true;
#NV_BEFORE = 0.3;  % antes
#NV_AFTER = 0.0;   % después


##### MFCC #####
# Params for mfcc calculation
NUM_MFCC = 13
##### MFCC #####

COLUMNS_NAMES_SPEAKER = ["speaker", "device"] # define los nombres de los parámetros separados por '_', uno de ellos tiene que ser speaker


#%% Definición de descriptores
FEATURES_TO_CALCULATE = ["mfcc","delta","deltaDelta"]

#% Los descriptores que se pueden calcular:
#% "mfcc"                mfcc
#% "delta"               delta mfcc
#% "deltaDelta"          delta delta
#% "vlhr"                6 mfcc con frecuencia de corte 400,500,...,900 Hz
#% "formants"            3 formantes frecuencia f1, f2, f3 Hz
#% "formantsBW"          3 ancho de banda formantes Hz
#% "formantsDist"        3 distancia entre formantes f1-f2, f1-f3, f2-f3 Hz
#% "f0"                  3 frecuencia fundamental f0, 2xf0, 3xf0
#% "intensity"           1 intensidad en dB como Praat


# Tiers del textGrid, posiciones y significado
TIER_WORDS = 0      # Palabra (no necesario)
TIER_PHONES = 1     # Fonemas
TIER_NLCE = 2       # Nasalancia del fonema
TIER_RMS = 3        # RMS del intervalo del fonema



## Ajustes previos a la ejecución
if not os.path.exists(RESULT_FOLDER):
    os.mkdir(RESULT_FOLDER)

AUDIO_PATH = os.path.join(os.getcwd(),AUDIO_FOLDER,AUDIO_DATABASE)
RESULT_PATH = os.path.join(os.getcwd(),RESULT_FOLDER,RESULT_DATABASE)
RESULT_PATH_XLSX = RESULT_PATH.replace('.csv','.xlsx')

MFCC_NAMES = [f"mfcc{x}" for x in range(0,NUM_MFCC)]
DELTA_NAMES = [f"delta{x}" for x in range(0,NUM_MFCC)]
DELTA_DELTA_NAMES = [f"deltaDelta{x}" for x in range(0,NUM_MFCC)]

In [2]:
from scipy.io import wavfile
import numpy as np
import librosa          # https://librosa.org/doc/latest/index.html
from pandas import DataFrame

import textgrid
from pathlib import Path
import pandas as pd 
import numpy as np
import os

import csv

from scipy.io.wavfile import write
from scipy.io import wavfile 

from python_speech_features import mfcc     # https://python-speech-features.readthedocs.io/en/latest/
from python_speech_features import delta

import scipy.io.wavfile as wav

In [3]:
def check_wav_textgrid(audio_folder):
    ''' 
    Comprueba que la carpeta audio_folder contiene archivos .wav y .textgrid en el mismo número.
    Devuelve dos listas con los archivos ordenados 
    '''

    audio_list = [os.path.join(audio_folder,file) for file in os.listdir(audio_folder) if file.endswith('.wav')]
    textgrid_list = [os.path.join(audio_folder,file) for file in os.listdir(audio_folder) if file.endswith('.TextGrid')]

    if not audio_list: # check if empty
        print(f"No hay archivos .wav en la carpeta: {audio_folder}")
        print("Introduce lo archivos .wav y vuelve a ejecutar el scipt.")
        exit(1)
    elif not textgrid_list: # check if empty
        print(f"No hay archivos .TextGrid en la carpeta: {audio_folder}")
        print("Introduce lo archivos .TextGrid y vuelve a ejecutar el scipt.")
        exit(1)
    elif len(audio_list) != len(textgrid_list): 
        print(f"No hay el mismo número de archivos .wav que .TextGrid en la carpeta: {audio_folder}")
        print("Cada archivo .wav tiene que venir acompañado de su archivo .TextGrid. Introduce los archivos que falten y vuelve a ejecutar el script.")
        exit(1)
    else:
        print(f"Número de archivos .wav y .TextGrid correcto, continúa la ejecución del script.")

    return sorted(audio_list), sorted(textgrid_list)


def get_speakeraudio_file(audio_path):
    '''
    Genera un dataframe con la información que se puede extraer del nombre del usuario. 
    La información tiene que estar separada por '_' y el nombre de cada parámetro viene definido en COLUMNS_NAMES_SPEAKER
    '''

    speaker_list = (os.path.split(audio_path)[-1]).split('_') # (head,tail)
    speaker_list[-1] = speaker_list[-1].replace('.wav','')
    speaker_df = pd.DataFrame([speaker_list], columns = COLUMNS_NAMES_SPEAKER) 

    return speaker_df

In [4]:
## MAIN

# def main():

# Se comprueba que la carpeta AUDIO_PATH contiene archivos .was y .textgrid
audio_list, textgrid_list = check_wav_textgrid(AUDIO_PATH)


Número de archivos .wav y .TextGrid correcto, continúa la ejecución del script.


In [5]:
# Se comprueba si existe la base de datos que se va a generar
print(f"Base de datos a generar: {RESULT_PATH}")

if os.path.exists(RESULT_PATH):
    print("Existe la base de datos")

    # nomBDstr=str(nombreBaseDatos)
    # print("La BD existe")
    # featuresTodosIntervalos=pd.read_csv(nombreBaseDatos)
    # listaLocutores=calculaLocutores(featuresTodosIntervalos)
    # print("Lista de locutores analizados anteriormente:")
    # for l in listaLocutores:
    #     print(l)
    # totalVentanas=len(featuresTodosIntervalos['Row'])
    # print("Número de ventanas en la BD original: ",totalVentanas)
else:
    print("La base de datos NO existe. Se crea una nueva.")
    # listaLocutores=[]
    # featuresTodosIntervalos=[]
    # totalVentanas=0
    # feature_list = []
    #compute_audio_features()

    # https://stackoverflow.com/questions/13784192/creating-an-empty-pandas-dataframe-and-then-filling-it

# ANTES DE CALCULAR LOS ELEMENTOS HAY QUE COMPROBAR QUE FEATURES_TO_CALCULATE NO ESTÁ VACÍA


Base de datos a generar: d:\OneDrive\nasalance\resultadosBaseDatos\pruebas_andres_22_oct.csv
Existe la base de datos


In [7]:
#def compute_audio_features():
'''
Calcula todos los descriptores
'''

# Variables de entrada
speakers_list = []
features_already_generated = [] # descriptores ya generados, por ejecuciones del bucle o por otra base de datos

# Variables para la generación
features_calculated_global = [] # descriptores calculados al finalizar el bucle for

for audio_file, textgrid_file in zip(audio_list,textgrid_list):
    #print(f"{audio_file} y {textgrid_file}")
    features_calculated_speaker = [] # descriptores calculados para este speaker

    # Datos que se generan para cada audio 
    speaker_df = get_speakeraudio_file(audio_file)

    # Comprueba si el speaker se encuentra en la base de datos
    spk_loop = speaker_df["speaker"].values[0]
    if spk_loop in speakers_list:
        print(f"Locutor {spk_loop} ya se encuentra en la base de datos. Pasamos al siguiente.")
    else:
        print(f"Locutor {spk_loop} NUEVO. Se calculan los descriptores.")

        # Se lee el audio
        samplerate, audio_data = wavfile.read(audio_file)


        ## Primero se calculan los descriptores
        # MFCC
        if 'mfcc' in FEATURES_TO_CALCULATE:
            mfcc_feat = mfcc(signal=np.array(audio_data, dtype=np.single), samplerate=samplerate, 
                winlen=WIN_LENGTH*1e-3, winstep=(WIN_LENGTH-WIN_OVERLAP)*1e-3, numcep=NUM_MFCC, 
                nfilt=26, nfft=1024, lowfreq=0, highfreq=None, preemph=0.97, ceplifter=22, appendEnergy=True, winfunc=np.hamming)

            features_calculated_speaker.append(mfcc_feat)

        # Delta
        if ('mfcc' in FEATURES_TO_CALCULATE) and ('delta' in FEATURES_TO_CALCULATE):
            delta_feat = delta(mfcc_feat, 2)

            features_calculated_speaker.append(delta_feat)

        # Delta delta
        if ('mfcc' in FEATURES_TO_CALCULATE) and ('delta' in FEATURES_TO_CALCULATE) and ('deltaDelta' in FEATURES_TO_CALCULATE):
            delta_delta_feat = delta(mfcc_feat, 2)
            
            features_calculated_speaker.append(delta_delta_feat)


        ## Datos del locutor
        for elem in reversed(speaker_df.columns): # orden inverso para ir metiéndolas al principio, como una pila
            features_calculated_speaker.insert(0, np.full((features_calculated_speaker[0].shape[0], 1), speaker_df[elem]))
        # OJO: SI AQUÍ NO ESTÁ DEFINIDO MFCC FALLA ¿QUEREMOS ALGÚN CASO EN EL QUE NO ESTÉ DEFINIDO MFCC?


        ## Clase del frame
        # POR COMPLETAR


        ## Se concatenan y se convierten en dataframe
        features_calculated_speaker = np.concatenate(features_calculated_speaker, axis=1) # se concatenan por columnas [A B C D]
        features_calculated_global.append(features_calculated_speaker)
    #end if
# end for

## Se guardan como .csv y .xlsx
features_calculated_global = np.concatenate(features_calculated_global, axis=0) # se concatenan por filas [A 
                                                                                 #                          B]

features_df = pd.DataFrame(features_calculated_global, columns = [list(speaker_df.columns) + MFCC_NAMES + DELTA_NAMES + DELTA_DELTA_NAMES][0])

print(f"\nGuardando datos en {RESULT_PATH_XLSX}")
#with pd.ExcelWriter(RESULT_PATH_XLSX) as writer:  
#    features_df.to_excel(writer, index=False, engine='openpyxl')
features_df.to_csv(RESULT_PATH, encoding='utf-8', index=False)  

print(f"\nFIN calculo descriptores")

Locutor nas20001 NUEVO. Se calculan los descriptores.
Locutor nas20002 NUEVO. Se calculan los descriptores.
Locutor nas20003 NUEVO. Se calculan los descriptores.
Locutor nas20004 NUEVO. Se calculan los descriptores.
Locutor nas20005 NUEVO. Se calculan los descriptores.
Locutor nas20007 NUEVO. Se calculan los descriptores.
Locutor nas20008 NUEVO. Se calculan los descriptores.
Locutor nas20010 NUEVO. Se calculan los descriptores.
Locutor nas20011 NUEVO. Se calculan los descriptores.
Locutor nas20013 NUEVO. Se calculan los descriptores.
Locutor nas20014 NUEVO. Se calculan los descriptores.
Locutor nas20015 NUEVO. Se calculan los descriptores.
Locutor nas20016 NUEVO. Se calculan los descriptores.
Locutor nas20017 NUEVO. Se calculan los descriptores.
Locutor nas20018 NUEVO. Se calculan los descriptores.
Locutor nas20019 NUEVO. Se calculan los descriptores.
Locutor nas20020 NUEVO. Se calculan los descriptores.
Locutor nas20021 NUEVO. Se calculan los descriptores.
Locutor nas20022 NUEVO. Se c

In [8]:
features_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 177785 entries, 0 to 177784
Data columns (total 41 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   speaker       177785 non-null  object
 1   device        177785 non-null  object
 2   mfcc0         177785 non-null  object
 3   mfcc1         177785 non-null  object
 4   mfcc2         177785 non-null  object
 5   mfcc3         177785 non-null  object
 6   mfcc4         177785 non-null  object
 7   mfcc5         177785 non-null  object
 8   mfcc6         177785 non-null  object
 9   mfcc7         177785 non-null  object
 10  mfcc8         177785 non-null  object
 11  mfcc9         177785 non-null  object
 12  mfcc10        177785 non-null  object
 13  mfcc11        177785 non-null  object
 14  mfcc12        177785 non-null  object
 15  delta0        177785 non-null  object
 16  delta1        177785 non-null  object
 17  delta2        177785 non-null  object
 18  delta3        177785 non

In [9]:
features_df.head()

,speaker,device,mfcc0,mfcc1,mfcc2,mfcc3,mfcc4,mfcc5,mfcc6,mfcc7,...,deltaDelta3,deltaDelta4,deltaDelta5,deltaDelta6,deltaDelta7,deltaDelta8,deltaDelta9,deltaDelta10,deltaDelta11,deltaDelta12
0,nas20001,iphone,9.166181,-23.231655,-6.049424,-4.11619,-4.747366,-3.538267,-3.089952,4.674421,...,1.376144,1.008055,0.730503,0.165444,-0.993729,4.364613,-3.331235,-1.778063,-5.437171,-4.6544
1,nas20001,iphone,8.59443,-18.461415,-7.247664,0.486281,-1.830832,6.670384,8.372136,11.618029,...,2.471942,1.061426,-1.755029,-2.882962,-1.769244,4.048242,-1.982144,1.527778,-2.659333,-3.866646
2,nas20001,iphone,8.277615,-19.190404,-8.235758,0.463294,-1.16536,-4.990079,-7.993775,-3.76603,...,1.975056,2.281428,0.871749,-0.584941,0.682196,4.469443,2.034723,1.755428,-0.857891,-2.640976
3,nas20001,iphone,7.866211,-13.83648,-5.748606,5.953777,-1.23124,-11.587506,-15.052852,0.048428,...,1.615844,1.963988,1.422236,-0.30003,2.610091,4.281639,2.853788,-0.504205,0.57849,1.694189
4,nas20001,iphone,7.982168,-13.16407,-6.478522,3.025342,6.359977,9.949424,5.697837,13.870203,...,1.832513,1.681403,3.00055,2.012544,3.130542,3.067337,5.592782,2.972192,2.164943,1.095929


In [14]:
# LIBROSA
f_mfcc = librosa.feature.mfcc(y=np.array(audio_data, dtype=np.single), sr=samplerate, n_mfcc=NUM_MFCC, 
                            win_length=win_length_samples, hop_length=(win_length_samples-win_overlap_samples))

print(f_mfcc.shape)

(14, 9761)


In [17]:
mfcc_feat = mfcc(signal=np.array(audio_data, dtype=np.single), samplerate=samplerate, 
                winlen=WIN_LENGTH*1e-3, winstep=(WIN_LENGTH-WIN_OVERLAP)*1e-3, numcep=NUM_MFCC, 
                nfilt=26, nfft=1024, lowfreq=0, highfreq=None, preemph=0.97, ceplifter=22, appendEnergy=True, winfunc=np.hamming)

print(mfcc_feat.shape)

(9761, 14)


In [22]:
print(f_mfcc[:,0])
print(mfcc_feat[0,:])

[ 3.7847467e+02  3.4444511e+01  2.2362719e+01  1.5456270e+01
  1.0243723e+01  8.4755859e+00  7.3981838e+00  3.4129996e+00
 -8.0048013e-01 -2.6222309e-01 -9.6232146e-01  5.0894299e+00
  8.5226669e+00  7.7294846e+00]
[  9.16618058 -23.23165494  -6.04942365  -4.11618962  -4.74736627
  -3.53826707  -3.08995233   4.67442104 -23.19542051  11.88455044
  -8.76536719  14.29689517  12.92363629   1.83140325]


In [15]:
WIN_LENGTH*1e-3

0.025

In [ ]:
#  Empiezan los cálculos para cada archivo
for file in audiosAsica:
	fileStr=str(file)
	contador=1

	ruta, archivoAudio = os.path.split(file)
	textGridStr=fileStr.replace(".wav",".TextGrid")
	rate,sig = wavfile.read(file)

	if archivoAudio.endswith(".wav"):
		archivoAudio=archivoAudio[:-4]
	print("Nuevo locutor: ", archivoAudio)
	if (archivoAudio in listaLocutores):
		print("-- Locutor repetido. Pasamos al siguiente")
		continue
	if (os.path.isfile(textGridStr)):
		grid=textgrid.TextGrid.fromFile(textGridStr)
		rutaT, archivoT = os.path.split(textGridStr)
		print("-- Leído el textGrid: ",archivoT)
	else:
		rutaT, archivoT = os.path.split(textGridStr)
		print("No se ha podido abrir el textGrid: ",textGridStr)
		print("-- Se ignora el wav. Pasamos al siguiente")
		continue


	# Bucle para calcular ventanas de cada clase
	numV_OC=0
	numV_OV=0
	numV_NC=0
	numV_NV=0
	numV_SL=0

	# grid=textgrid.TextGrid.fromFile(textGridStr)
	
	nIntervalos=len(grid[tierRMS].intervals)-1
	for intervalo in (range(nIntervalos)):
		phone=grid[tierPhones][intervalo].mark
		rms=int(grid[tierRMS][intervalo].mark)
		if len(grid[tierNlce][intervalo].mark)>0:
			nlceText=grid[tierNlce][intervalo].mark
			nlceText=nlceText.split('.')[0]
			nlce=int(nlceText)
		else:
			nlce=0
		sc=subclas(phone, nlce, rms)
		min=grid[tierRMS][intervalo].minTime
		max=grid[tierRMS][intervalo].maxTime
		duracion=max-min
		nv=int(round((duracion-0.015)/0.01))
		if sc=="SL":
			numV_SL=numV_SL+nv
		elif sc=="OV":
			numV_OV=numV_OV+nv
		elif sc=="OC":
			numV_OC=numV_OC+nv
		elif sc=="NV":
			numV_NV=numV_NV+nv
		elif sc=="NC":
			numV_NC=numV_NC+nv
		else:
			numV_NC=numV_NC

	# zz por aquí voy. Ahora tengo que controlar el número de ventanas de cada clase. y no seguir si ha llegado al límite
	maximos=[numV_NC, numV_OC, numV_NV, numV_OV,numV_SL]
	# print(maximos)
	elementoMenor=np.argmin(maximos)
	elementoMayor=np.argmax(maximos)

	if balanceado:
		maxVentanas=maximos[elementoMenor]
		if numClases==3:
			maxVentanasSil=maxVentanas*2
		elif numClases==5:
			maxVentanasSil=maxVentanas
		else:
			print("Error: solo puede hacer tres o cinco clases")
			exit()
	else:
		maxVentanas=maximos[elementoMayor]


	print("Max. NC: ", numV_NC)
	print("Max. OC: ", numV_OC)
	print("Max. NV: ", numV_NV)
	print("Max. OV: ", numV_OV)
	print("Max. SL: ", numV_SL)
	print("Max ventanas: ",maxVentanas)

	numV_NC=0
	numV_OC=0
	numV_NV=0
	numV_OV=0
	numV_SL=0

	nIntervalos=len(grid[tierRMS].intervals)-1
	for intervalo in (range(nIntervalos)):
		if (intervalo > -1):
			min=grid[tierRMS][intervalo].minTime
			max=grid[tierRMS][intervalo].maxTime
			duracion=max-min
			nv=int(round((duracion-0.015)/0.01))
			beginFragment=int(round(min*rate))
			endFragment=int(round(max*rate))
			fragmento=sig[beginFragment:endFragment]
			phone=grid[tierPhones][intervalo].mark
			rms=int(grid[tierRMS][intervalo].mark)
			if len(grid[tierNlce][intervalo].mark)>0:
				nlceText=grid[tierNlce][intervalo].mark
				nlceText=nlceText.split('.')[0]
				nlce=int(nlceText)
			else:
				nlce=0
			
			sc=subclas(phone, nlce, rms)
			
			# Si ya tenemos el máximo de ventanas de una cat, pasamos al sig. intervalo
			if sc=="SL":
				if numV_SL > maxVentanasSil:
					continue
				else:
					numV_SL=numV_SL+nv
			if sc=="OC":
				if numV_OC > maxVentanas:
					continue
				else:
					numV_OC=numV_OC+nv
			if sc=="OV":
				if numV_OV > maxVentanas:
					continue
				else:
					numV_OV=numV_OV+nv
			if sc=="NC":
				if numV_NC > maxVentanas:
					continue
				else:
					numV_NC=numV_NC+nv
			if sc=="NV":
				if numV_NV > maxVentanas:
					continue
				else:
					numV_NV=numV_NV+nv


			if duracion>0 and len(featuresTodosIntervalos)==0 and sc!="":
				features=allFeatures(rate,fragmento) # Archivo de Audio
				features=defineColumnaRow(features, sc, archivoAudio, contador)
				featuresTodosIntervalos=features
				contador=contador+len(features)
				# print("Ventana inicial: ",contador,"Sc:",sc, " Inicio: ",min," Fin: ",max)

			elif duracion>0 and len(featuresTodosIntervalos)>0 and sc!="":				
				features=allFeatures(rate,fragmento) # Archivo de Audio
				features=defineColumnaRow(features, sc, archivoAudio, contador)
				contador=contador+len(features)
				featuresTodosIntervalos = pd.concat([featuresTodosIntervalos, features])
				# print("Ventana inicial: ",contador,"Sc:",sc, " Inicio: ",min," Fin: ",max)
			else:
				a=1
				#print("Intervalo no anotado: se ignora. Inicio: ",min," Fin: ",max)
	totalVentanas=totalVentanas+contador
	ventanasNuevas=ventanasNuevas+contador
	print("-- Ventanas procesadas en el último audio: ",contador)
	print("-- Ventanas procesadas en total: ", totalVentanas)
		


In [10]:
not len(audio_list)

False

In [3]:
def moving_window(audio, width=1000, overlap=400):
    audio = audio.tolist()
    return [audio[i : i + width] for i in range(0, len(audio), width-overlap)]

In [4]:
samplerate, audio_data = wavfile.read(AUDIO_PATH)
print(len(audio_data))
# Transform length and overlap from ms to samples
win_length_samples = int(WIN_LENGTH * samplerate / 1000)
win_overlap_samples = int(WIN_OVERLAP * samplerate / 1000)
print(win_length_samples)
print(win_overlap_samples)

audio_win =  moving_window(audio_data, width=win_length_samples, overlap=win_overlap_samples)
print(type(audio_win))

665571
1102
441
<class 'list'>


In [7]:
# def compute_features(audio, samplerate):
audio = audio_win

feat_audio = []

#for chunk in audio:
chunk = audio[0]
if 'mfcc' in FEATURES:
    # Calculate mfcc for audio
    librosa.feature.mfcc(y=audio_win, sr=samplerate, n_mfcc=NUM_MFCC, dct_type=2, norm='ortho', lifter=0)

elif 'delta' in FEATURES:
    # do nothing
    8


SyntaxError: invalid character '¶' (U+00B6) (600687401.py, line 10)

In [36]:
f_mfcc = librosa.feature.mfcc(y=np.array(audio_data, dtype=np.single), sr=samplerate, n_mfcc=NUM_MFCC, 
                            win_length=win_length_samples)

print(f_mfcc)
print(len(f_mfcc[0]))                        
print(f_mfcc.shape)

[[ 4.86895996e+02  5.04622528e+02  4.94123230e+02 ...  3.78741302e+02
   3.73278259e+02  3.86671753e+02]
 [ 8.32832870e+01  8.94101410e+01  8.41661148e+01 ...  1.53775215e+01
   9.10860062e+00  2.74301033e+01]
 [-1.21022902e+01 -2.49979515e+01 -2.47444077e+01 ...  1.08155813e+01
   7.34319687e+00  2.38158760e+01]
 ...
 [-5.59439242e-01 -7.57081389e-01  2.49762490e-01 ... -1.51912427e+00
  -1.48723435e+00  1.65443528e+00]
 [-9.99189377e+00 -7.82266235e+00 -5.94357204e+00 ... -1.91025078e+00
  -1.23540735e+00  9.63437200e-01]
 [-1.47784491e+01 -1.05038767e+01 -9.53153038e+00 ... -2.38252163e+00
  -1.03475046e+00  8.92013788e-01]]
1300
(14, 1300)


In [29]:
len(np.array(audio_data, dtype=np.single))

665571